<a href="https://colab.research.google.com/github/Elwing-Chou/ml1206/blob/main/inception.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
model = InceptionV3(include_top=False, input_shape=(224, 224, 3))
model.summary()

In [ ]:
from tensorflow.keras.utils import plot_model
plot_model(model, show_shapes=True, to_file="inception.png")

In [8]:
# 右一: 64個1x1conv
w = model.get_layer("conv2d_5").get_weights()[0]
w.shape

(1, 1, 192, 64)

In [9]:
# 右二: 模糊化 -> 32個1x1conv
w = model.get_layer("conv2d_11").get_weights()[0]
w.shape

(1, 1, 192, 32)

In [10]:
# 右三: 48個1x1conv(減低計算量)->64個5x5conv
w = model.get_layer("conv2d_6").get_weights()[0]
print(w.shape)
w = model.get_layer("conv2d_7").get_weights()[0]
print(w.shape)

(1, 1, 192, 48)
(5, 5, 48, 64)


In [11]:
# 右四: 64個1x1conv->96個3x3conv->96個3x3conv
w = model.get_layer("conv2d_8").get_weights()[0]
print(w.shape)
w = model.get_layer("conv2d_9").get_weights()[0]
print(w.shape)
w = model.get_layer("conv2d_10").get_weights()[0]
print(w.shape)

(1, 1, 192, 64)
(3, 3, 64, 96)
(3, 3, 96, 96)
